In [ ]:
import pandas as pd
import plotly.express as px

def plot_dhrystone_histogram(csv_file):
    # Load the data
    df = pd.read_csv(csv_file)

    # Create grouped bar chart
    fig = px.bar(
        df, 
        x="runs", 
        y="cycles", 
        color="core", 
        barmode="group",  # Group bars
        facet_col="compiler",  # Separate subplots by compiler
        labels={"cycles": "CPU Cycles", "runs": "Dhrystone Runs", "core": "Core"}
    )

    # Improve layout
    fig.update_layout(
        title="Dhrystone Benchmark - Cycles vs Runs",
        xaxis_title="Dhrystone Runs",
        yaxis_title="CPU Cycles",
        bargap=0.9,  # Adjust bar spacing
        bargroupgap=0.05,  # Adjust group spacing
        width=600,  # Reduce overall figure width
        height=500  # Adjust height to balance aspect ratio
    )

    fig.show()
    fig.write_image("Dhrystone.pdf")

# Example usage
plot_dhrystone_histogram("dhrystone_normalized.csv")


In [ ]:
import pandas as pd
import plotly.express as px

def plot_dhrystone_histogram(csv_file):
    # Load the data
    df = pd.read_csv(csv_file)

    # Create a new column to combine core & compiler info
    df["core_compiler"] = df["core"] + " (" + df["compiler"] + ")"

    # Create a grouped bar chart
    fig = px.bar(
        df, 
        x="runs", 
        y="cycles", 
        color="core_compiler",  # Color by combined core & compiler info
        barmode="group",  # Grouped bars
        labels={
            "cycles": "CPU Cycles", 
            "runs": "Dhrystone Runs", 
            "core_compiler": "Core (Compiler)"
        }
    )

    # Adjust layout to control bar width
    fig.update_layout(
        title="Dhrystone Benchmark - Cycles vs Runs (Grouped by Core & Compiler)",
        xaxis_title="Dhrystone Runs",
        yaxis_title="CPU Cycles",
        bargap=0.9,  # Adjust bar spacing within groups
        bargroupgap=0.1,  # Adjust spacing between different groups
        width=800,  # Adjust figure width
        height=500
    )

    fig.show()

# Example usage
plot_dhrystone_histogram("dhrystone.csv")


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("dhrystone.csv")

# Create a pivot table to access gcc cycle values for normalization
pivot = df.pivot(index=["core", "runs"], columns="compiler", values="cycles")

# Normalize cycles so that gcc = 1 and llvm is relative to gcc
pivot["llvm_normalized"] = pivot["llvm"] / pivot["gcc"]
pivot["gcc_normalized"] = 1  # Keep gcc values as 1

# Convert back to a long-form DataFrame
normalized_df = pivot.reset_index().melt(id_vars=["core", "runs"], 
                                         value_vars=["gcc_normalized", "llvm_normalized"], 
                                         var_name="compiler", 
                                         value_name="cycles")

# Fix compiler names to match original (remove "_normalized" suffix)
normalized_df["compiler"] = normalized_df["compiler"].str.replace("_normalized", "")

# Display the new DataFrame
print(normalized_df)
normalized_df.to_csv("dhrystone_normalized.csv")


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("dhrystone.csv")

#pivot = df.pivot(index=["core", "runs"], columns="compiler", values="cycles")
#print(pivot)

#df = pd.read_csv(io.StringIO(data))

# Find the GCC cycles for each core
gcc_cycles = df[df['compiler'] == 'gcc'].set_index('core')['cycles']

# Create a new DataFrame with normalized cycles
normalized_df = df.copy()
normalized_df['normalized_cycles'] = normalized_df.apply(
    lambda row: row['cycles'] / gcc_cycles[row['core']] ,
    axis=1
)

print(gcc_cycles)
print(normalized_df)

In [ ]:
import pandas as pd

file1 = "conv2d_test.csv"
file2 = "conv2d_test_lca.csv"

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Concatenate the DataFrames
df = pd.concat([df1, df2], ignore_index=True)

lca_cycles = df[df['arch'] == 'LCA'].set_index('latency')['cycles'].to_dict()

# Save to a new CSV file (optional)
#merged_df.to_csv("merged_file.csv", index=False)

# Create a dictionary mapping latency to LCA cycles
df['normalized_cycles'] = df.apply(
    lambda row: row['cycles'] / lca_cycles.get(row['latency'], row['cycles']),
    axis=1
)


print(df)